<a href="https://colab.research.google.com/github/XetmanX/Kaggle/blob/main/RuCodeP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from keras import models,layers,optimizers
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras_preprocessing.image import ImageDataGenerator

np.random.seed(8)
tf.random.set_seed(8)

In [2]:
#data https://drive.google.com/drive/folders/1bAQCZXxm-dXiHTrxteMYibOr_echdcKZ

data = pd.read_csv('/content/train.csv')
data.head()

,file_nm,label
0,1.jpg,0
1,2.jpg,2
2,3.jpg,1
3,4.jpg,2
4,5.jpg,3


In [3]:
!unzip '/content/data.zip'

Archive:  /content/data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/535.jpg      
  inflating: data/train/1971.jpg     
  inflating: data/train/1021.jpg     
  inflating: data/train/21.jpg       
  inflating: data/train/2172.jpg     
  inflating: data/train/497.jpg      
  inflating: data/train/1001.jpg     
  inflating: data/train/1685.jpg     
  inflating: data/train/19.jpg       
  inflating: data/train/1328.jpg     
  inflating: data/train/2326.jpg     
  inflating: data/train/298.jpg      
  inflating: data/train/1027.jpg     
  inflating: data/train/700.jpg      
  inflating: data/train/1953.jpg     
  inflating: data/train/391.jpg      
  inflating: data/train/372.jpg      
  inflating: data/train/1048.jpg     
  inflating: data/train/1740.jpg     
  inflating: data/train/1584.jpg     
  inflating: data/train/1336.jpg     
  inflating: data/train/37.jpg       
  inflating: data/train/1618.jpg     
  inflating: data/train/2232.jpg     
  inflating: dat

In [6]:
datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input, 
                             validation_split=0.25, horizontal_flip=True)

shape = (380, 380)
batch_size = 32

train_generator=datagen.flow_from_dataframe(
    dataframe=data,
    directory="/content/data/train",
    x_col="file_nm",
    y_col='label',
    subset="training",
    batch_size=batch_size,
    seed=8,
    shuffle=True,
    class_mode="raw",
    color_mode='rgb',
    target_size=shape)


valid_generator=datagen.flow_from_dataframe(
    dataframe=data,
    directory="/content/data/train",
    x_col="file_nm",
    y_col='label',
    subset="validation",
    batch_size=batch_size,
    seed=8,
    shuffle=True,
    class_mode="raw",
    color_mode='rgb',
    target_size=shape)


test = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory='/content/data',
    target_size=shape,
    color_mode="rgb",
    classes=['test'],
    class_mode="sparse",
    batch_size=batch_size,
    shuffle=False,
    seed=8,
    save_to_dir=None,
    save_prefix="",
    save_format="jpg",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

Found 1775 validated image filenames.
Found 591 validated image filenames.
Found 2367 images belonging to 1 classes.


In [7]:
base_model=tf.keras.applications.EfficientNetB4(include_top=False, weights="imagenet", input_tensor=None, input_shape=(380, 380, 3), pooling='avg')
base_model.summary()

71696384/71686520 [==============================] - 1s 0us/step
Model: "efficientnetb4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 380, 380, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 380, 380, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization_1[0][0]            
____________________

In [8]:
model = models.Sequential()
model.add(base_model)
model.get_layer('efficientnetb4').trainable = False 
model.add(layers.Dropout(0.2))
model.add(layers.Dense(4, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 1792)              17673823  
_________________________________________________________________
dropout (Dropout)            (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 7172      
Total params: 17,680,995
Trainable params: 7,172
Non-trainable params: 17,673,823
_________________________________________________________________


In [10]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_generator, epochs=15, validation_data=valid_generator)

Epoch 1/15
56/56 [==============================] - 91s 1s/step - loss: 0.2384 - accuracy: 0.9448 - val_loss: 0.1515 - val_accuracy: 0.9594
Epoch 2/15
56/56 [==============================] - 72s 1s/step - loss: 0.1095 - accuracy: 0.9701 - val_loss: 0.1267 - val_accuracy: 0.9611
Epoch 3/15
56/56 [==============================] - 72s 1s/step - loss: 0.0838 - accuracy: 0.9752 - val_loss: 0.1134 - val_accuracy: 0.9662
Epoch 4/15
56/56 [==============================] - 71s 1s/step - loss: 0.0734 - accuracy: 0.9792 - val_loss: 0.1105 - val_accuracy: 0.9662
Epoch 5/15
56/56 [==============================] - 72s 1s/step - loss: 0.0667 - accuracy: 0.9780 - val_loss: 0.1019 - val_accuracy: 0.9712
Epoch 6/15
56/56 [==============================] - 72s 1s/step - loss: 0.0530 - accuracy: 0.9842 - val_loss: 0.1046 - val_accuracy: 0.9679
Epoch 7/15
56/56 [==============================] - 73s 1s/step - loss: 0.0533 - accuracy: 0.9876 - val_loss: 0.1036 - val_accuracy: 0.9712
Epoch 8/15
56/56 [==

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(valid_generator, epochs=1)

19/19 [==============================] - 32s 970ms/step - loss: 0.0961 - accuracy: 0.9729


In [ ]:
#model.save('/content/model')

In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('/content/model')

In [12]:
ans = model.predict(test)
ans_file = pd.DataFrame([np.argmax(ans[i]) for i in range(ans.shape[0])])
ans_file.to_csv('P.csv',index=False, header=False)